In [72]:
%pip install openai graphrag pandas requests python-dotenv langchain numpy tiktoken matplotlib scikit-learn pyyaml pydantic instructor
from IPython.display import clear_output ; clear_output()

In [61]:
from dotenv import load_dotenv
import os
load_dotenv()

is_azure = (
  os.getenv("AZURE_OPENAI_ENDPOINT", default="") != "" and
  os.getenv("OPENAI_API_KEY", default="") == ""
)

GPT_4_O_MODEL_NAME = os.getenv("GPT_4_O_MODEL_NAME", default="gpt-4o")
TEXT_EMBEDDING_3_LARGE_MODEL_NAME = os.getenv("TEXT_EMBEDDING_3_LARGE_MODEL_NAME", default="text-embedding-3-large")

if is_azure:
  AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
  AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
  AZURE_OPENAI_API_VERSION = "2024-05-01-preview"
  from openai import AzureOpenAI
  oai = AzureOpenAI(azure_endpoint=AZURE_OPENAI_ENDPOINT, api_key=AZURE_OPENAI_API_KEY, api_version=AZURE_OPENAI_API_VERSION)
else:
  OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
  from openai import OpenAI
  oai = OpenAI(api_key=OPENAI_API_KEY)

In [62]:
import requests
import os

if not os.path.exists('data'): os.makedirs('data')

if not os.path.exists('data/french_revolution.md'):
  french_revolution = requests.get("https://r.jina.ai/https://en.wikipedia.org/wiki/French_Revolution").text.split('\nSee also')[0]
  with open('data/french_revolution.md', 'w') as f:
    f.write(french_revolution)
else:
  with open('data/french_revolution.md', 'r') as f:
    french_revolution = f.read()

print(french_revolution[:123])

Title: French Revolution

URL Source: https://en.wikipedia.org/wiki/French_Revolution

Published Time: 2001-10-18T00:19:10Z


In [63]:
from langchain.text_splitter import MarkdownTextSplitter
import pandas as pd

if not os.path.exists('data/embeddings.parquet'):
  embeddings = pd.DataFrame(columns=['Topic', 'Text', 'Embedding'])

  splitter = MarkdownTextSplitter(chunk_size=500, chunk_overlap=250)

  chunks = splitter.split_text(french_revolution)
  chunk_embeddings = oai.embeddings.create(
    input=chunks,
    model=TEXT_EMBEDDING_3_LARGE_MODEL_NAME
  )
  for i, chunk in enumerate(chunks):
    try:
      topic = oai.chat.completions.create(
        model=GPT_4_O_MODEL_NAME,
        messages=[
          {
            "role": "system",
            "content": ("Read the user-provided text carefully and output its topic as a short sentence. "
                        "For example: 'Key events in the life of George Washington', 'Inflation in the Weimar republic'. "
                        "Do not add any additional text such as punctuation, markup, or quoting. Only output the topic.")},
          {"role": "user", "content": chunk}
        ],
        max_tokens=23,
        temperature=0.5,
      ).choices[0].message.content
    except Exception:
      pass
    embeddings.loc[len(embeddings)] = [topic or chunk[:23], chunk, chunk_embeddings.data[i].embedding]
  embeddings.to_parquet('data/embeddings.parquet')
else:
  embeddings = pd.read_parquet('data/embeddings.parquet')

embeddings

,Topic,Text,Embedding
0,French Revolution,Title: French Revolution\n\nURL Source: https:...,"[-0.02485630474984646, 0.0026923343539237976, ..."
1,Key events of the French Revolution,"The Storming of the Bastille, 14 July 1789\n\n...","[-0.005255864467471838, 0.0037901357281953096,..."
2,The French Revolution and its impact on modern...,The French Revolution[a] was a period of polit...,"[-0.027185581624507904, -0.016875529661774635,..."
3,Causes and early events of the French Revolution,Its causes are generally agreed to be a combin...,"[-0.020989766344428062, -0.024809090420603752,..."
4,Key events in the early stages of the French R...,which was converted into a National Assembly i...,"[-0.009496822953224182, -0.017249926924705505,..."
...,...,...,...
296,Alphonse Aulard's contributions to the study o...,evidence.[266][267] Alphonse Aulard (1849–1928...,"[-0.0005632034735754132, 0.021342121064662933,..."
297,Marxist socio-economic analysis of the French ...,Socio-economic analysis and a focus on the exp...,"[-0.003857595380395651, 0.007019456941634417, ..."
298,Alfred Cobban's critique of Jacobin-Marxist in...,Alfred Cobban challenged Jacobin-Marxist socia...,"[0.004117575474083424, 0.00744161382317543, -0..."
299,Political decisions and radicalization during ...,"In their 1965 work, La Revolution française, F...","[0.017132386565208435, 0.012499384582042694, -..."


In [64]:
import numpy as np
import tiktoken

def cosine_similarity(vector1, vector2):
  dot_product = np.dot(vector1, vector2)
  norm1 = np.linalg.norm(vector1)
  norm2 = np.linalg.norm(vector2)
  similarity = dot_product / (norm1 * norm2)
  return similarity

tokenizer = tiktoken.encoding_for_model('gpt-4o')

def embeddings_search(query, max_tokens=10000, k=100, min_similarity=0.2):
  query_embedding = oai.embeddings.create(
    input=[query],
    model=TEXT_EMBEDDING_3_LARGE_MODEL_NAME
  ).data[0].embedding
  results = embeddings.copy()
  results['Similarity'] = results['Embedding'].apply(lambda x: cosine_similarity(x, query_embedding))
  results = results.sort_values(by='Similarity', ascending=False).head(k)
  results = results[results['Similarity'] >= min_similarity]
  results['Tokens'] = results['Text'].apply(lambda txt: len(tokenizer.encode(txt)))
  while results['Tokens'].sum() > max_tokens:
    results = results[:-1]
  return results

In [65]:
DEFAULT_RESPONSE_TYPE = 'Summarize and explain in 1-2 paragraphs with bullet points using at most 300 tokens'
DEFAULT_MAX_CONTEXT_TOKENS = 10000

from graphrag.query.structured_search.global_search.reduce_system_prompt import REDUCE_SYSTEM_PROMPT

def ask_embeddings(query, response_type=DEFAULT_RESPONSE_TYPE):
  results = embeddings_search(query, max_tokens=DEFAULT_MAX_CONTEXT_TOKENS)
  response = oai.chat.completions.create(
    model=GPT_4_O_MODEL_NAME,
    messages=[
      {
        "role": "system",
        "content": REDUCE_SYSTEM_PROMPT.format(
          response_type=response_type,
          report_data="---\n---\n".join(results['Text'].tolist()),
        ),
      },
      {"role": "user", "content": query}
    ],
    max_tokens=4000,
    temperature=0.5,
  ).choices[0].message.content
  return response

In [66]:
from IPython.display import Markdown

result = ask_embeddings('Who was Robespierre and what was his role in the French revolution?')

Markdown(result)

### Maximilien Robespierre and His Role in the French Revolution

**Maximilien Robespierre** was a prominent figure in the French Revolution, known for his radical stances and influential role during the Reign of Terror. Below are the key points summarizing his involvement and impact:

#### Early Involvement
- **Political Alignment**: Robespierre was a radical Montagnard, opposing the criteria for "active citizens" which disenfranchised many of the Parisian proletariat [Data: Reports (2, 8, +more)].
- **Support Base**: He gained substantial support among the working class and was a key figure in the Jacobin club [Data: Reports (1, 2, 6, +more)].

#### Reign of Terror
- **Committee of Public Safety**: Robespierre was a leading member of the Committee of Public Safety, which assumed control after the assassination of Marat and the suspension of the 1793 Constitution [Data: Reports (11, 1, +more)].
- **Law of 22 Prairial**: This law, pushed by Robespierre, denied "enemies of the people" the right to defend themselves, leading to a significant increase in executions [Data: Reports (27, 7, +more)].
- **Execution of Rivals**: He oversaw the execution of political rivals, including Georges Danton and Camille Desmoulins, consolidating his power but also creating many enemies [Data: Reports (5, 20, +more)].

#### Downfall and Death
- **Opposition and Arrest**: Robespierre's refusal to name alleged conspirators led to confusion and fear. His opponents struck first, leading to his arrest and subsequent execution on 28 July 1794, marking the end of the Reign of Terror [Data: Reports (1, 7, +more)].
- **Impact on the Revolution**: His death allowed the repeal of the Law of 22 Prairial and the reinstatement of surviving Girondists. The Jacobin Club was also closed and banned [Data: Reports (7, 27, +more)].

Robespierre remains a controversial figure, symbolizing both the revolutionary zeal and the extreme measures taken during the French Revolution.

In [67]:
result = ask_embeddings('Timeline of the French revolution')

Markdown(result)

### Timeline of the French Revolution

- **1789**
  - **May:** Financial crisis and social distress lead to the convocation of the Estates General.
  - **June:** Estates General converts into the National Assembly.
  - **July 14:** Storming of the Bastille, symbolizing the start of the Revolution.
  - **August:** Abolition of feudalism and state control over the Catholic Church [Data: Reports (51, 55, 58, 62, 65, +more)].

- **1790**
  - **July 14:** Fête de la Fédération celebrates the fall of the Bastille, showing temporary unity [Data: Reports (48, 49, 65)].

- **1791**
  - **September:** Constitution of 1791 establishes a constitutional monarchy; National Assembly dissolves [Data: Reports (61, 89)].

- **1792**
  - **April:** Outbreak of the French Revolutionary Wars.
  - **August 10:** Storming of the Tuileries Palace; monarchy effectively suspended.
  - **September:** Proclamation of the French First Republic [Data: Reports (100, 101, 106)].

- **1793**
  - **January:** Execution of Louis XVI.
  - **June:** Constitution of 1793 introduced but suspended due to internal conflict.
  - **September:** Reign of Terror begins; Law of Suspects enacted [Data: Reports (108, 114, 118, 119)].

- **1794**
  - **July:** Execution of Robespierre marks the end of the Reign of Terror [Data: Reports (121, 122)].

- **1795**
  - **November:** Establishment of the Directory as the new government [Data: Reports (121)].

- **1799**
  - **November 9:** Coup of 18 Brumaire; establishment of the French Consulate, marking the end of the Revolutionary period [Data: Reports (159)].

### Key Outcomes

- Abolition of the Ancien Régime and establishment of constitutional monarchy.
- Proclamation of the French First Republic.
- Initiation of the Reign of Terror and execution of Louis XVI.
- French Revolutionary Wars.
- Establishment of the French Consulate [Data: Reports (1, 6, 7, 106, 159, +more)].

In [68]:
import yaml

if not os.path.exists('data/graphrag'):
  !python -m graphrag.index --init --root data/graphrag

with open('data/graphrag/settings.yaml', 'r') as f:
  settings_yaml = yaml.load(f, Loader=yaml.FullLoader)
settings_yaml['llm']['model'] = GPT_4_O_MODEL_NAME
settings_yaml['llm']['api_key'] = AZURE_OPENAI_API_KEY if is_azure else OPENAI_API_KEY
settings_yaml['llm']['type'] = 'azure_openai_chat' if is_azure else 'openai_chat'
settings_yaml['embeddings']['llm']['api_key'] = AZURE_OPENAI_API_KEY if is_azure else OPENAI_API_KEY
settings_yaml['embeddings']['llm']['type'] = 'azure_openai_embedding' if is_azure else 'openai_embedding'
settings_yaml['embeddings']['llm']['model'] = TEXT_EMBEDDING_3_LARGE_MODEL_NAME
if is_azure:
  settings_yaml['llm']['api_version'] = AZURE_OPENAI_API_VERSION
  settings_yaml['llm']['deployment_name'] = GPT_4_O_MODEL_NAME
  settings_yaml['llm']['api_base'] = AZURE_OPENAI_ENDPOINT
  settings_yaml['embeddings']['llm']['api_version'] = AZURE_OPENAI_API_VERSION
  settings_yaml['embeddings']['llm']['deployment_name'] = TEXT_EMBEDDING_3_LARGE_MODEL_NAME
  settings_yaml['embeddings']['llm']['api_base'] = AZURE_OPENAI_ENDPOINT

with open('data/graphrag/settings.yaml', 'w') as f:
  yaml.dump(settings_yaml, f)

if not os.path.exists('data/graphrag/input'):
  os.makedirs('data/graphrag/input')
  !cp data/french_revolution.md data/graphrag/input/french_revolution.txt
  !python -m graphrag.index --root ./data/graphrag

🚀 Reading settings from data/graphrag/settings.yaml
⠹ GraphRAG Indexer 
⠹ GraphRAG Indexer les loaded (0 filtered) ━━━━━━ 100% 0:00:… 0:00:…
├── Loading Input (text) - 1 files loaded (0 filtered) ━━━━━━ 100% 0:00:… 0:00:…
⠹ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ━━━━━━ 100% 0:00:… 0:00:…
⠹ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ━━━━━━ 100% 0:00:… 0:00:…
⠹ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ━━━━━━ 100% 0:00:… 0:00:…
⠴ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ━━━━━━ 100% 0:00:… 0:00:…
⠴ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ━━━━━━ 100% 0:00:… 0:00:…
⠴ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ━━━━━━ 100% 0:00:… 0:00:…
⠴ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filtered) ━━━━━━ 100% 0:00:… 0:00:…
⠴ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (0 filter

In [69]:
import subprocess

def ask_graph(query, response_type=DEFAULT_RESPONSE_TYPE):
  env = os.environ.copy() | {
    'GRAPHRAG_GLOBAL_SEARCH_MAX_TOKENS': str(DEFAULT_MAX_CONTEXT_TOKENS),
  }
  command = [
    'python', '-m', 'graphrag.query',
    '--root', './data/graphrag',
    '--method', 'local',
    '--response_type', response_type,
    query,
  ]
  output = subprocess.check_output(command, universal_newlines=True, env=env)
  return output.split('Search Response: ')[1]

In [70]:
from IPython.display import Markdown

result = ask_graph('Who was Robespierre and what was his role in the French revolution?')

Markdown(result)

### Maximilien Robespierre: Key Figure in the French Revolution

**Overview:**
Maximilien Robespierre was a central figure during the French Revolution, known for his radical views and significant influence within the revolutionary government.

**Roles and Contributions:**

- **Leading Figure:**
  - Robespierre was a prominent leader within the Jacobins and the Montagnard faction [Data: Entities (52); Relationships (263, 269)].
  - He was a key member of the Committee of Public Safety, which held significant power during the Revolution [Data: Entities (52); Relationships (136)].

- **Reign of Terror:**
  - Robespierre's actions and policies were central to the Reign of Terror, a period marked by mass executions of perceived enemies of the revolution [Data: Entities (52); Relationships (97)].
  - He was involved in the execution of notable figures such as Georges Danton and Hébert [Data: Entities (52, 186); Relationships (265, 267)].

- **Cult of the Supreme Being:**
  - Robespierre led the establishment of the Cult of the Supreme Being, a revolutionary cult that faced ridicule and opposition [Data: Entities (249); Relationships (268)].

- **Political Dynamics:**
  - He opposed property qualifications for voting and standing for office, advocating for broader democratic participation [Data: Entities (52); Relationships (269, 280)].
  - Robespierre's radical policies and claims of conspiracy led to intense political rivalries and his eventual downfall [Data: Entities (52); Relationships (271, 275)].

- **Downfall and Execution:**
  - The Convention authorized his arrest, leading to his execution on 28 July 1794, marking the end of his influential role in the Revolution [Data: Entities (52); Relationships (275)].

**Key Relationships:**

- **Georges Danton:**
  - Despite being close friends and political allies, Robespierre saw Danton's moderate stance as destabilizing, leading to Danton's execution [Data: Entities (186); Relationships (265, 266)].

- **Saint-Just:**
  - Saint-Just was a close ally of Robespierre, and both were executed together, highlighting their intertwined political careers [Data: Entities (218); Relationships (264)].

**Conclusion:**
Robespierre was a pivotal figure in the French Revolution, whose radical policies and leadership during the Reign of Terror left a lasting impact on the course of the revolution and French history. His eventual downfall underscores the volatile nature of revolutionary politics.


In [71]:
result = ask_graph('Timeline of the French revolution')

Markdown(result)

# Timeline of the French Revolution

## Key Events

- **1789**
  - **May 5**: Estates-General convened at Versailles, marking the beginning of the French Revolution [Data: Entities (62)].
  - **June 17**: National Assembly formed by the Third Estate.
  - **July 14**: Storming of the Bastille, a pivotal event symbolizing the uprising against the monarchy [Data: Entities (1); Relationships (0, 9, 59)].
  - **October 5-6**: March to Versailles by women demanding bread and political reforms [Data: Entities (397)].

- **1791**
  - **June 20-21**: Flight to Varennes, Louis XVI's failed attempt to escape Paris [Data: Entities (3, 80); Relationships (81)].
  - **September 3**: Constitution of 1791 adopted, establishing a constitutional monarchy.

- **1792**
  - **April 20**: France declares war on Austria, beginning the French Revolutionary Wars [Data: Entities (7); Relationships (6)].
  - **August 10**: Insurrection of 10 August 1792, leading to the fall of the monarchy [Data: Entities (14)].
  - **September 21**: Proclamation of the French First Republic [Data: Entities (5); Relationships (4)].

- **1793**
  - **January 21**: Execution of Louis XVI, marking a significant turning point [Data: Entities (3, 80); Relationships (2)].
  - **September 5**: Beginning of the Reign of Terror, characterized by mass executions and political repression [Data: Entities (6); Relationships (5, 20)].

- **1794**
  - **July 28**: Execution of Robespierre, ending the Reign of Terror [Data: Entities (6)].

- **1795**
  - **October 5**: Napoleon suppresses a royalist uprising in Paris [Data: Entities (22); Relationships (200)].

- **1799**
  - **November 9**: Coup of 18 Brumaire, leading to the establishment of the French Consulate and marking the end of the French Revolution [Data: Entities (7); Relationships (7)].

## Impact and Legacy

- The French Revolution led to the abolition of the Ancien Régime, the establishment of the French First Republic, and significant social, political, and economic changes in France and beyond [Data: Entities (0, 21, 89); Relationships (28, 146)].
- It inspired international movements and reforms, influencing political thought and actions worldwide [Data: Entities (20); Relationships (48, 181, 361)].

This timeline highlights the major events and transformations during the French Revolution, showcasing its profound impact on France and the world.
